In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Paths
input_folder = "/content/drive/MyDrive/skin_Dataset/Skin_Dataset/Segmented pictures"
output_folder = "/content/drive/MyDrive/skin_Dataset/Skin_Dataset/O-results"
result_folder = "/content/drive/MyDrive/skin_Dataset/Skin_Dataset/bgraabx"

# Create result folder if it does not exist
os.makedirs(result_folder, exist_ok=True)

# Image size
IMG_HEIGHT, IMG_WIDTH = 256, 256

# Function to load and preprocess images
def load_images(folder, is_label=False):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE if is_label else cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        if is_label:
            img = img / 255.0  # Normalize to [0, 1]
            img = np.expand_dims(img, axis=-1)
        else:
            img = img / 255.0  # Normalize to [0, 1]
        images.append(img)
    return np.array(images)

# Load data
X = load_images(input_folder)
y = load_images(output_folder, is_label=True)

# Define U-Net model
def build_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u1 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u1 = concatenate([u1, c3])
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(u1)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u2 = concatenate([u2, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u3 = concatenate([u3, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs, outputs)
    return model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Build and compile model
model = build_unet((IMG_HEIGHT, IMG_WIDTH, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, validation_split=0.1, epochs=10, batch_size=2,callbacks=[reduce_lr, early_stopping])

# Predict and save results
for i, img in enumerate(X):
    prediction = model.predict(np.expand_dims(img, axis=0))[0]
    prediction = (prediction > 0.5).astype(np.uint8) * 255
    save_path = os.path.join(result_folder, f"result_{i + 1}.png")
    cv2.imwrite(save_path, prediction)


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 99s 10s/step - accuracy: 0.7493 - loss: 0.6917 - val_accuracy: 0.8285 - val_loss: 0.6706 - learning_rate: 1.0000e-04
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 140s 10s/step - accuracy: 0.7853 - loss: 0.6826 - val_accuracy: 0.8285 - val_loss: 0.6214 - learning_rate: 1.0000e-04
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 91s 10s/step - accuracy: 0.7996 - loss: 0.6520 - val_accuracy: 0.8285 - val_loss: 0.5179 - learning_rate: 1.0000e-04
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 91s 10s/step - accuracy: 0.8071 - loss: 0.5856 - val_accuracy: 0.8285 - val_loss: 0.4708 - learning_rate: 1.0000e-04
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 150s 11s/step - accuracy: 0.7847 - loss: 0.5532 - val_accuracy: 0.8285 - val_loss: 0.4834 - learning_rate: 1.0000e-04
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 134s 10s/step - accuracy: 0.8180 - loss: 0.4468 - val_accuracy: 0.8285 - val_loss: 0.4940 - learning_rate: 1.0000e-04
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.8386 - loss: 0.42